# UTN - Data Engineering - Modulo 2 - TP2
## Procesamiento de datos y almacenamiento OLAP.

En el TP1 se extrajo de una API la temperatura, presion y velocidad del viento almacenandolos en forma cruda.
A continuacion en este TP se van a realizar una seria de transformaciones (a pedido del cliente) a estos datos.

El Cliente solicito trabajar por ciudad, tal cual fueron almacenados los datos crudos en sistema Parquet.
Los datos de cada ciudad necesitan ser trabajados para luego pode analizarlos. Las transformaciones solicitadas fueron las siguientes:

1. Eliminar los datos duplicados, dejando el ultimo valor encontrado.
2. Completar si existen los valores nulos con 99999.
3. Poner las fechas en formato time.
4. Generar nuevas columnas con el 'Año', 'Mes', 'Hora' de los datos obtenidos.
5. Renombrar las columnas a un formato standar seleccionado por el cliente.
6. Armar tres tablas de la siguiente forma:
    1. Tabla 1 - Los datos completos procesados anteriormente.
    2. Tabla 2 - Los datos que corresponden a un rango horario diurno.
    3. Tabla 3 - Los datos que corresponden a un rango harario nocturno.

Las tablas las silocita el cliente de esta forma porque necesita analizar el comportamiento del viento en superficie (como fn. de la temp y la pres) segun sea de dia o de noche. A su vez necesita analizar la data comleta para comparar resultados.


### Se instalan las librerias principales

In [ ]:
#Se instalan las librerias utilizadas para la conexioncon la DB y se actualiza Pandas
!pip install psycopg2-binary
!pip install sqlalchemy
!pip install pandas
!pip install fastparquet

### Se importan las Librerias

Importo Pandas para el manejo de Data Frames.

Importo datetime para el manejo de las Fechas y las horas. Sobre todo en el split (%Y, %M, %H) que solicita el cliente.

Importo sqlalchemy para la coneccion con la Base de Datos y las tablas.


In [1]:
import pandas as pd
import os.path
from datetime import datetime
from sqlalchemy import create_engine, MetaData, Table, Column, String, Float, Integer, Date, Time
import psycopg2

from sqlalchemy import create_engine
from sqlalchemy.sql import text

/Users/eduardo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/eduardo/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


### Seccion de Funciones soporte.

In [2]:
#Creo una funcion para verificar que la ciudad exista y tenga un archivo data en bronce.
#Si el archivo existe se carga en un DataFrame.
#Si no existe se emite un msg avisando que la ciudad no existe.
def loadData(city):
    directoryPath = 'weather/' + city + '/bronce/data.parquet'

    if os.path.exists(directoryPath):
        df = pd.read_parquet(directoryPath)
        print(f"Los datos de la ciudad '{city}' fueron cargados con exito.")
    else:
        print(f"La ciudad '{city}' no existe.")
        df = pd.DataFrame()
    return df

#Funcion para crear las tablas en la base de datos Postgresql y actualizarlas incrementalmente con los df.
#Esta funcion se conecta a la base de datos con las credenciales y crea, si no exite una tabla
#con el nombre que se la pasa a la funcion y se actualiza con el df pasado por la funcion

def createDataBaseTable(tableName, df_pass): #se pasa el nombre de la tabla deseada y el data frame.
    #Conexion con la base de datos
    # Definimos el string de conexión
    host = "postgresql-eduardofarina-self-01.a.aivencloud.com"
    port = 27648
    database = "defaultdb"
    username = "avnadmin"
    password = "AVNS_HnesWmqFpCrMBE477T8"

    conn_string = f"postgresql://{username}:{password}@{host}:{port}/{database}"

    engine = create_engine(conn_string)

    create_table_query = text(f"""CREATE TABLE IF NOT EXISTS {tableName} (
                            id SERIAL PRIMARY KEY,
                            time TIMESTAMP,
                            temp FLOAT,
                            speed FLOAT,
                            pres FLOAT,
                            year VARCHAR(50),
                            month VARCHAR(50),
                            hour INTEGER);
                         """)
    with engine.connect() as conn, conn.begin():
        conn.execute(create_table_query)
    print(f"Tabla {tableName} creada exitosamente en PostgreSQL ")
    
    # Consulta SQL para insertar datos en la tabla
    # Se utiliza el df que se pasa en la funcion.
    with engine.connect() as conn, conn.begin():
        df_pass.to_sql(
            tableName,
            schema="public",
            con=conn,
            if_exists="append",
            method="multi",
            index=False
        )
    print("Conexión a PostgreSQL cerrada")

### Seccion Recupero de la informacion extraida de la API.

En este paso se recupera la informacion guardada en formato Parquet para una ciudad en particular.

In [3]:
#En esta variable se solicita la ciudad en particular que deseamos levantar la informacion.
city = 'Pico Truncado'
#Se envia la ciudad a la Funcion de verificacion y carga de los datos
df = loadData(city)
#Se muestra en encabezado del df.
df.head()

Los datos de la ciudad 'Pico Truncado' fueron cargados con exito.


,time,temperature_2m,wind_speed_10m,surface_pressure
index,,,,
0,2024-01-01T00:00,22.6,28.6,976.6
1,2024-01-01T01:00,20.0,23.1,977.4
2,2024-01-01T02:00,18.2,17.0,978.3
3,2024-01-01T03:00,16.7,13.3,978.7
4,2024-01-01T04:00,15.3,9.1,979.3


### Seccion de Transformacion de la informacion

In [4]:
#Se eliminan los datos duplicados dejando el ultimo bajado.
#No se realiza un sort, porque lo que se busca es dejar el ultimo dato duplicado que se haya bajado de la API.
#Ej. un dia se bajan xx cantidad de datos hasta una fecha. Y luego otro dia despues se bajan las mismas fechas.
#Como son pronosticos del clima, el Cliente considera que es de esperar que los ultimos datos tengan mayor certeza.

df.drop_duplicates(subset=None, keep='last', inplace=True, ignore_index=False)

In [5]:
#Si existen se reemplazan los valores nulos por 99999.

df.fillna(value=99999, method=None, axis=None, inplace=True)

In [6]:
#Aca se convierte la columna FechaTHora en formato Fecha Hora para cargar en la Base de Datos.

df['time'] = pd.to_datetime(df['time'])

#Se hace un split de esa fecha en Año, Mes y hora de cada dato registrado.
#Se crean nuevas columnas.
df['year'] = df['time'].dt.strftime('%Y')
df['month'] = df['time'].dt.strftime('%m')
df['hour'] = df['time'].dt.strftime('%H').astype(int)


In [7]:
#Se renombran las columnas en un formato standar a pedido del cliente.
df.rename(columns={'temperature_2m': 'temp', 'wind_speed_10m': 'speed', 'surface_pressure': 'pres'}, inplace=True)


In [8]:
#Se crean dos nuevos DataFrames uno con los datos correspondientes al horario diurno.
#Otro con los datos correspondientes a la noche.

#Split de horarios Diurnos
df_day = df.loc[df['hour'].between(8, 19)]

#Por la negativa (~), split de horarios nocturnos
df_night = df.loc[~df['hour'].between(8, 19)]

In [26]:
df_day

,time,temp,speed,pres,year,month,hour
1,2024-01-01 01:00:00,20.0,23.1,977.4,2024,01,1
2,2024-01-01 02:00:00,18.2,17.0,978.3,2024,01,2
25,2024-01-02 01:00:00,15.9,22.9,977.9,2024,01,1
26,2024-01-02 02:00:00,15.1,22.1,977.9,2024,01,2
49,2024-01-03 01:00:00,25.1,18.3,975.7,2024,01,1
...,...,...,...,...,...,...,...
2018,2024-03-25 02:00:00,13.7,18.6,981.8,2024,03,2
2041,2024-03-26 01:00:00,18.1,24.1,972.6,2024,03,1
2042,2024-03-26 02:00:00,17.3,25.4,972.6,2024,03,2
2065,2024-03-27 01:00:00,15.4,15.3,978.2,2024,03,1


### En esta Seccion se crean, si no existen las tablas en la base de datos Postgresql
### Y se actualizan con los df que solicita el cliente.

Por favor Guido, ten en cuenta mi limitacion en la programacion.
Seguramente este codigo se puede mejorar mucho.
Ademas la subida a la base de datos demora bastante.

In [28]:
#Se llama a la funcion 3 veces para crear las tres tablas solicitadas.
#Tabla con los datos completos
createDataBaseTable('eduardofarina_datos', df)

#Tabla con los datos split, solo horarios Diurnos
createDataBaseTable('eduardofarina_day', df_day)

#Tabla con los datos split, solo horarios nocturnos
createDataBaseTable('eduardofarina_night', df_night)

Tabla eduardofarina_3 creada exitosamente en PostgreSQL 
Tabla eduardofarina_3 actualizada con datos exitosamente
Conexión a PostgreSQL cerrada


### Fin de TP-2